In [2]:
using DataFrames
using ForwardDiff
x0=[0.1,0.2,0.1];
alpha=0.4;
beta=0.5;


Data= readtable("binary.csv")
l=length(Data[:,1]);
U=zeros(l,3);
U[:,1]=Data[:,2];
U[:,2]=Data[:,3];
U[:,3] = ones(l);

function L(x,u)
    return -u[1]*(x[1]*u[2]+x[2]*u[2]+u[3])+(1-u[1])*(log(1+exp(x[1]*u[2]+x[2]*u[2]+u[3])))
end

G(x,u) = ForwardDiff.gradient(x-> L(x,u))(x)
H(x,u) = ForwardDiff.hessian(x->L(x,u))(x)
function Logistic(x)
    m=0;
    for i=1:l
        u=[Data[i,1],Data[i,2],Data[i,3]];
    m=L(x,u)+m;
    end
    return m
end


function GradL(x)
    m=0;
    for i = 1:l
    u=[Data[i,1],Data[i,2],Data[i,3]];
    m=G(x,u)+m;
    end
    return m
end

function Hes(x)
    m=0;
    for i = 1:l
    u=[Data[i,1],Data[i,2],Data[i,3]];
    m=H(x,u)+m;
    end
    return m
end   
Grd=[0];
x0=[0.1,0.2,0.1];
function newtmin(Logistic, GradL,Hes, x0; maxit=500, optTol=1e-14)
# Minimize a function f using Newton’s method.
# obj: a function that evaluates the objective value,
# gradient, and Hessian at a point x, i.e.,
# (f, g, H) = obj(x)
# x0: starting point.
# maxIts (optional): maximum number of iterations.
# optTol (optional): optimality tolerance based on
# ||grad(x)|| <= optTol*||grad(x0)||
    
    f_0 = Logistic(x0);
    grad_0 = GradL(x0);
    Hes_0 = Hes(x0);
    Hes_0 = Hes_0 - min(eigmin(Hes_0)-1e-3,0)*eye(length(x0));

    x = x0;
    f_x = f_0;
    grad_x = grad_0;   
    Hes_x = Hes_0;
    Grd=[norm(grad_x)];
    #Counts the iteratives.
    int = 1;
    while optTol * norm(grad_0) <= norm(grad_x)
        int=int+1;

        
        # if the iteratives number is greater than maxit, break down.
        if int >= maxit
            break
        end
        
    # applying netwon method
    
    # determine the direction.
        dx = - Hes_x \  grad_x ;
        t=1;
        bt=0;
    print
        at=grad_x' * dx;

        while Logistic(x+t*dx) >= Logistic(x)+alpha*t*at[1]
        
                t= t * beta;
        end # back tracking
        
        
        x=x+dx*t;
    
        f_x=Logistic(x);
        grad_x = GradL(x);
        Hes_x = Hes(x);
        Hes_x=Hes_x - min(eigmin(Hes_x)-1e-3,0)*eye(length(x0));
        Grd=append!(Grd,[norm(grad_x)]);
    
    
    
    end # while newton M
    
    return(x, int,Grd)   
end   # function

(x,int,Grd)= newtmin(Logistic, GradL,Hes, x0; maxit=500, optTol=1e-14)
println("x_minimizer=",x,"\nL_value=",Logistic(x), "\ngrad_L=", GradL(x),"\nnum of iteration=", int)
println(Grd)

x_minimizer=[-0.052820315743646,0.04717968429434554,0.1]
L_value=216.74454490695285
grad_L=[1.1129941412946209e-10,1.1129941412946209e-10,0.0]
num of iteration=7
[110138.95223761664,110089.7663115755,81119.85716240198,10879.494358459839,154.27585792843195,0.01554000104413747,1.5740114094606498e-10]


In [1]:
Pkg.add("ForwardDiff")
Pkg.add("DataFrames")


INFO: Nothing to be done
INFO: Nothing to be done
